In [29]:
import pandas as pd
import sklearn
import random, os 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from fairness_evals import positive_rates, true_postive_rates, true_negative_rate, false_postive_rates

# Data

In [30]:
data_old = pd.read_csv(os.path.join('Processed_data', 'adult_occp.csv'))

In [31]:
data_old.keys()

Index(['age', 'workclass', 'education', 'marital.status', 'occupation', 'race',
       'sex', 'hours.per.week', 'income'],
      dtype='object')

In [32]:
data_old

,age,workclass,education,marital.status,occupation,race,sex,hours.per.week,income
0,82,Private,HS-grad,Widowed,Exec-managerial,White,Female,18,0
1,54,Private,7th-8th,Divorced,Machine-op-inspct,White,Female,40,0
2,41,Private,Some-college,Separated,Prof-specialty,White,Female,40,0
3,34,Private,HS-grad,Divorced,Other-service,White,Female,45,0
4,38,Private,10th,Separated,Adm-clerical,White,Male,40,0
...,...,...,...,...,...,...,...,...,...
30157,22,Private,Some-college,Never-married,Protective-serv,White,Male,40,0
30158,27,Private,Assoc,Married-civ-spouse,Tech-support,White,Female,38,0
30159,40,Private,HS-grad,Married,Machine-op-inspct,White,Male,40,1
30160,58,Private,HS-grad,Widowed,Adm-clerical,White,Female,40,0


In [33]:
DATA = data_old 
# DATA.drop(columns=['occupation'])
DATA_NAME = 'old' #! don't forget to change this accordingly 
FEAT_OF_INT = ['education', 'marital.status', 'sex', 'income'] #The features we are interested in
DATA.head()

,age,workclass,education,marital.status,occupation,race,sex,hours.per.week,income
0,82,Private,HS-grad,Widowed,Exec-managerial,White,Female,18,0
1,54,Private,7th-8th,Divorced,Machine-op-inspct,White,Female,40,0
2,41,Private,Some-college,Separated,Prof-specialty,White,Female,40,0
3,34,Private,HS-grad,Divorced,Other-service,White,Female,45,0
4,38,Private,10th,Separated,Adm-clerical,White,Male,40,0


## Encoder for categorical values

In [34]:
from sklearn.preprocessing import OneHotEncoder
cat_feats = list(DATA.select_dtypes("object").keys())  #names of all categorical features
# cat_feats.remove('income') #keep these values as is 
print(f"{len(cat_feats)} categorical features found: {cat_feats}")

cat_feat_encoder = OneHotEncoder(sparse_output=False, drop='if_binary').set_output(transform="pandas")
cat_feat_encoder = cat_feat_encoder.fit(DATA[cat_feats])

6 categorical features found: ['workclass', 'education', 'marital.status', 'occupation', 'race', 'sex']


## Split train, dev and test

In [37]:
train, dev = train_test_split(DATA, random_state= 1989)
# test = data_2018
print(f"Train size: {len(train)}, dev size: {len(dev)}")

Train size: 22621, dev size: 7541


In [38]:
x_train = train.drop(columns=['income'])
y_train = train['income']

x_dev  = dev.drop(columns=['income'])
y_dev  = dev['income']

# x_test  = test.drop(columns=['income'])
# y_test  = test['income']

In [39]:
from Dataverwerking.data_transform import cat_to_one_hot
x_train_encoded = cat_to_one_hot(x_train, cat_feats, cat_feat_encoder)
x_dev_encoded = cat_to_one_hot(x_dev, cat_feats, cat_feat_encoder)
# x_test_encoded = cat_to_one_hot(x_test, cat_feats, cat_feat_encoder)

In [40]:
x_train_encoded

,age,hours.per.week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_10th,...,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Male
15483,48,40,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
16653,53,60,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
7635,28,40,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
16496,22,40,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
7664,32,60,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,56,50,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
15397,49,40,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4924,45,80,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
18180,40,40,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


# Logistic regression

**params logreg**
* Penalty (=L2)
* C (=1): regularization strenght -> set with dev set 
* solver: ?

In [41]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver = 'sag', max_iter= 1000).fit(x_train_encoded, y_train)

/Users/jip/Documents/Uni_master/Human-Centered ML/HCML_project/.venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## Dev evaluation

In [42]:
y_pred_dev= list(logreg.predict(x_dev_encoded))

logreg_results_dev = dev.copy() #get all the data of the dev set

logreg_results_dev = logreg_results_dev.filter(items=FEAT_OF_INT) #keep only relevant info
logreg_results_dev['y_pred'] = y_pred_dev
logreg_results_dev

,education,marital.status,sex,income,y_pred
15688,Bachelors,Divorced,Female,0,0
4719,HS-grad,Married,Male,1,1
12320,HS-grad,Married,Male,0,0
9038,HS-grad,Divorced,Male,0,0
22761,HS-grad,Married,Male,0,0
...,...,...,...,...,...
1489,Masters,Divorced,Male,1,0
13261,Masters,Married,Male,1,1
29103,11th,Married,Male,0,0
26107,Bachelors,Married,Male,0,1


In [43]:
logreg_acc_dev = accuracy_score(y_dev, y_pred_dev)
logreg_rec_dev = recall_score(y_dev, y_pred_dev)
logreg_pre_dev = precision_score(y_dev, y_pred_dev)
logreg_f1_dev = f1_score(y_dev, y_pred_dev)
logreg_eval_dev = pd.DataFrame({'accuracy': [logreg_acc_dev], 'recall': [logreg_rec_dev], 'precision': [logreg_pre_dev], 'f1': [logreg_f1_dev]}, index=['dev'])
# logreg_eval_dev.to_csv(os.path.join('Results', f'dev_{DATA_NAME}_eval.csv'),sep='\t')
logreg_eval_dev

,accuracy,recall,precision,f1
dev,0.832648,0.547812,0.704656,0.616413


In [44]:
logreg_fair_eval_dev = pd.DataFrame()
logreg_fair_eval_dev['positive rates'] = positive_rates(logreg_results_dev, 'sex')
logreg_fair_eval_dev['true positive rates'] = true_postive_rates(logreg_results_dev, 'sex')
logreg_fair_eval_dev['true negative rates'] = true_negative_rate(logreg_results_dev, 'sex')
logreg_fair_eval_dev['false positive rates'] = false_postive_rates(logreg_results_dev, 'sex')
# logreg_fair_eval_dev.to_csv(os.path.join('Results', f'dev_{DATA_NAME}_fair_eval.csv'),sep='\t')
logreg_fair_eval_dev


,positive rates,true positive rates,true negative rates,false positive rates
Female,0.050021,0.298932,0.982694,0.017306
Male,0.257321,0.592357,0.890766,0.109234


# DO NOT RUN FROM HERE ON

In [28]:
DATA = pd.read_csv(os.path.join('Processed_data', 'adult_occp.csv'))
# DATA.drop(columns=['occupation'])
DATA_NAME = 'old_occp' #! don't forget to change this accordingly 
FEAT_OF_INT = ['education', 'marital.status', 'sex', 'income'] #The features we are interested in
DATA.head()
cat_feats = list(DATA.select_dtypes("object").keys())  #names of all categorical features
# cat_feats.remove('income') #keep these values as is 
print(f"{len(cat_feats)} categorical features found: {cat_feats}")
best_f1 = 0 
best_acc = 0

for drop in ['if_binary', 'first']:
    cat_feat_encoder = OneHotEncoder(sparse_output=False, drop=drop).set_output(transform="pandas")
    cat_feat_encoder = cat_feat_encoder.fit(DATA[cat_feats])
    for solver in ['sag', 'saga']:
        for max_iter in range(0, 2001, 50):
            print(f"\nencoding: {drop}, solver: {solver}, max_iter: {max_iter}")
            train, dev = train_test_split(DATA, random_state= 1989)

            x_train = train.drop(columns=['income'])
            y_train = train['income']

            x_dev  = dev.drop(columns=['income'])
            y_dev  = dev['income']

            x_train_encoded = cat_to_one_hot(x_train, cat_feats, cat_feat_encoder)
            x_dev_encoded = cat_to_one_hot(x_dev, cat_feats, cat_feat_encoder)

            logreg = LogisticRegression(solver = solver, max_iter= max_iter).fit(x_train_encoded, y_train)
            
            y_pred_dev= list(logreg.predict(x_dev_encoded))

            logreg_results_dev = dev.copy() #get all the data of the dev set

            logreg_results_dev = logreg_results_dev.filter(items=FEAT_OF_INT) #keep only relevant info
            logreg_results_dev['y_pred'] = y_pred_dev

            logreg_acc_dev = accuracy_score(y_dev, y_pred_dev)
            if logreg_acc_dev>best_acc: best_acc = logreg_acc_dev
            logreg_rec_dev = recall_score(y_dev, y_pred_dev)
            logreg_pre_dev = precision_score(y_dev, y_pred_dev)
            logreg_f1_dev = f1_score(y_dev, y_pred_dev)
            if logreg_f1_dev>best_f1: best_f1 = logreg_f1_dev
            logreg_eval_dev = pd.DataFrame({'accuracy': [logreg_acc_dev], 'recall': [logreg_rec_dev], 'precision': [logreg_pre_dev], 'f1': [logreg_f1_dev]}, index=['dev'])
            # logreg_eval_dev.to_csv(os.path.join('Results', f'dev_{DATA_NAME}_eval.csv'),sep='\t')
            display(logreg_eval_dev)
print(best_f1, best_acc)
# BEST: encoding: if_binary, solver: sag, max_iter: 1000-1150 -> 0.6164133738601824 || 0.832648
#       encoding: if_binary, solver: saga, max_iter: 1950-2000

6 categorical features found: ['workclass', 'education', 'marital.status', 'occupation', 'race', 'sex']

encoding: if_binary, solver: sag, max_iter: 0


/Users/jip/Documents/Uni_master/Human-Centered ML/HCML_project/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,accuracy,recall,precision,f1
dev,0.754542,0.0,0.0,0.0



encoding: if_binary, solver: sag, max_iter: 50


/Users/jip/Documents/Uni_master/Human-Centered ML/HCML_project/.venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


...


encoding: first, solver: saga, max_iter: 1950


,accuracy,recall,precision,f1
dev,0.832516,0.545651,0.705307,0.615291



encoding: first, solver: saga, max_iter: 2000


,accuracy,recall,precision,f1
dev,0.832516,0.545651,0.705307,0.615291


0.6164133738601824 0.8331786235247315
